<a href="https://colab.research.google.com/github/YounSooKimTech/UN_user_migration_project/blob/main/User_Migartion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Preprocessing
- Extract country and title information from UNINFO
- Filter the dataset for specific position (Operation Management Team Chair, OMT Chair)
-

In [5]:
# Load the dataset from drive

from google.colab import drive

drive.mount("/content/gdrive")

import pandas as pd

df = pd.read_excel("/content/gdrive/MyDrive/Data/UN_USER/20231009_Result_BOS_UNINFO_user_list.xlsx", sheet_name = "BOS_UNINFO_user_list_key_roles")
df['UNINFO.User Roles'].fillna("", inplace=True)
df.head(3)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


,First_Name,Last_Name,Email,User_name,Access_To_BOS,State,Poste_title,UNINFO.id,UNINFO.email,UNINFO.firstname,...,UNINFO.status,UNINFO.username,UNINFO.isDmo,UNINFO.Organization abbreviation,UNINFO.Organization,UNINFO.User Roles,BOS_AF_user_info.COUNTRY,BOS_AF_user_info.AGENCY,BOS_AF_user_info.BOS_ACCOUNTABILITY,BOS_UNINFO_Head_of_RCO
0,Mickelle Antionette,HUGHES,mickelle.hughes@un.org,mickelle.hughes@un.org,1,ACTIVATED,NaN,127.0,mickelle.hughes@un.org,Mickelle,...,active,mickelle.hughes,0.0,UN RCO,United Nations Resident Coordinator Office,"Agency Focal Point - Malawi, CF Editor - Mala...",Malawi,UNS – RCOs,Head of RCO,True
1,Candra,SAMEKTO,c.samekto@ifad.org,c.samekto@ifad.org,1,ACTIVATED,Country Programme Manager,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,none,Afghanistan,IFAD,UNCT,False
2,Salette,BETTENCOURT,salette.bettencourt@cv.jo.un.org,salette.bettencourt@cv.jo.un.org,1,ACTIVATED,Operations Manager,4236.0,salette.bettencourt@cv.jo.un.org,Salette,...,active,salette.bettencourt,0.0,UNDP,United Nations Development Programme,"CF Viewer - Cabo Verde, Common Premise Champi...",Cabo Verde,UNDP,OMT Chair,False


In [6]:
# Make the title list

UNINFO_temp_titles = [s.split('-')[0].strip() for s in df["UNINFO.User Roles"]]
UNINFO_temp_titles_set = set(UNINFO_temp_titles)
UNINFO_temp_titles_set.remove("")
UNINFO_temp_titles_set.remove("none")
UNINFO_temp_titles_set_list = list(UNINFO_temp_titles_set)

BOS_titles = list(df["BOS_AF_user_info.BOS_ACCOUNTABILITY"].unique())

title_lists = BOS_titles + UNINFO_temp_titles_set_list
title_lists = list(set(title_lists))

def extract_titles(description):
    matched_titles = []
    for title in title_lists:
        if title in description:
            matched_titles.append(title)
    return ",".join(matched_titles) if matched_titles else None

df['UNINFO_titles'] = df['UNINFO.User Roles'].apply(extract_titles)
df['UNINFO_titles'] = df['UNINFO_titles'].str.strip()
#df["extracted_titles"].fillna("", inplace=True)

In [7]:
# Extract the country from UNINFO.User Roles
# The country list is from BOS Country

country = list(df["BOS_AF_user_info.COUNTRY"].unique())

def extract_countries(description):
    matched_countries = []
    for c in country:
        if c in description:
            matched_countries.append(c)
    return ",".join(matched_countries) if matched_countries else None

# Create a new column 'extracted_country' with the extracted countries
df['UNINFO_country'] = df['UNINFO.User Roles'].apply(extract_countries)
df['UNINFO_country'] = df['UNINFO_country'].str.strip()

#df["extracted_country"].fillna("", inplace=True)
#df["BOS_AF_user_info.COUNTRY"].fillna("", inplace=True)

In [8]:
! pip install pycountry

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 27.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for pycountry: filename=pycountry-22.3.5-py2.py3-none-any.whl size=10681832 sha256=1081b2727e407e4f762a53c84c1c2d614e45be9124ee6a68266bdbda1b387965
  Stored in directory: /root/.cache/pip/wheels/03/57/cc/290c5252ec97a6d78d36479a3c5e5ecc76318afcb241ad9dbe
Successfully built pycountry


In [9]:
import pycountry

def get_iso3_multiple(country_names):
    iso3_codes = []
    country_list = country_names.split(', ')
    for country_name in country_list:
        country = pycountry.countries.get(name=country_name)
        if country:
            iso3_codes.append(country.alpha_3)
    return ', '.join(iso3_codes)
df["BOS_ISO3"] = df["BOS_AF_user_info.COUNTRY"].apply(get_iso3_multiple)

df["UNINFO_ISO3"] = df["UNINFO_country"].fillna("").apply(get_iso3_multiple)

In [10]:
df.head()

,First_Name,Last_Name,Email,User_name,Access_To_BOS,State,Poste_title,UNINFO.id,UNINFO.email,UNINFO.firstname,...,UNINFO.Organization,UNINFO.User Roles,BOS_AF_user_info.COUNTRY,BOS_AF_user_info.AGENCY,BOS_AF_user_info.BOS_ACCOUNTABILITY,BOS_UNINFO_Head_of_RCO,UNINFO_titles,UNINFO_country,BOS_ISO3,UNINFO_ISO3
0,Mickelle Antionette,HUGHES,mickelle.hughes@un.org,mickelle.hughes@un.org,1,ACTIVATED,NaN,127.0,mickelle.hughes@un.org,Mickelle,...,United Nations Resident Coordinator Office,"Agency Focal Point - Malawi, CF Editor - Mala...",Malawi,UNS – RCOs,Head of RCO,True,"Agency Focal Point,IMS Team Leader,CF Survey E...",Malawi,MWI,MWI
1,Candra,SAMEKTO,c.samekto@ifad.org,c.samekto@ifad.org,1,ACTIVATED,Country Programme Manager,NaN,NaN,NaN,...,NaN,none,Afghanistan,IFAD,UNCT,False,None,None,AFG,
2,Salette,BETTENCOURT,salette.bettencourt@cv.jo.un.org,salette.bettencourt@cv.jo.un.org,1,ACTIVATED,Operations Manager,4236.0,salette.bettencourt@cv.jo.un.org,Salette,...,United Nations Development Programme,"CF Viewer - Cabo Verde, Common Premise Champi...",Cabo Verde,UNDP,OMT Chair,False,"Common Premise Champion,CF Viewer",Cabo Verde,CPV,CPV
3,Susan,MARTIN,susan.martin@wfp.org,susan.martin@wfp.org,1,ACTIVATED,Un Engagement Officer,5030.0,susan.martin@wfp.org,Susan,...,United Nations World Food Programme,"CF Editor - Kenya, CF Viewer - Kenya",Senegal,WFP,Agency HQ Focal Point,False,"CF Viewer,CF Editor",Kenya,SEN,KEN
4,Gaston,OSANGO,gaston.osango@undp.org,gaston.osango@undp.org,1,ACTIVATED,Operations Manager,6140.0,gaston.osango@undp.org,Gaston,...,United Nations Development Programme,Agency Focal Point - Democratic Republic of th...,Democratic Republic of the Congo,UNDP,OMT Working Group Chair,False,"Agency Focal Point,CF Viewer,CF Editor","Democratic Republic of the Congo,Congo",,


In [11]:
# Filter the position based on accoutnability

title_to_check = "OMT Chair"

df_filtered = df[df["BOS_AF_user_info.BOS_ACCOUNTABILITY"] == title_to_check]
df_filtered.sort_values(by="BOS_AF_user_info.COUNTRY")

print(df_filtered["BOS_AF_user_info.BOS_ACCOUNTABILITY"].value_counts(), "\n") #Yes! Only OMT chair is in the file

df_filtered.head(2)

OMT Chair    265
Name: BOS_AF_user_info.BOS_ACCOUNTABILITY, dtype: int64 



,First_Name,Last_Name,Email,User_name,Access_To_BOS,State,Poste_title,UNINFO.id,UNINFO.email,UNINFO.firstname,...,UNINFO.Organization,UNINFO.User Roles,BOS_AF_user_info.COUNTRY,BOS_AF_user_info.AGENCY,BOS_AF_user_info.BOS_ACCOUNTABILITY,BOS_UNINFO_Head_of_RCO,UNINFO_titles,UNINFO_country,BOS_ISO3,UNINFO_ISO3
2,Salette,BETTENCOURT,salette.bettencourt@cv.jo.un.org,salette.bettencourt@cv.jo.un.org,1,ACTIVATED,Operations Manager,4236.0,salette.bettencourt@cv.jo.un.org,Salette,...,United Nations Development Programme,"CF Viewer - Cabo Verde, Common Premise Champi...",Cabo Verde,UNDP,OMT Chair,False,"Common Premise Champion,CF Viewer",Cabo Verde,CPV,CPV
16,Kateryna,LYTVYNENKO,klytvynenko@unicef.org,klytvynenko@unicef.org,1,ACTIVATED,Operations Manager,5182.0,klytvynenko@unicef.org,Kateryna,...,United Nations Children's Fund,"CF Viewer - Angola, Common Premise Champion -...",Angola,UNICEF,OMT Chair,False,"Common Premise Champion,CF Viewer",Angola,AGO,AGO


In [12]:
# Extract the columns which is needed

df_result = df_filtered[["BOS_AF_user_info.COUNTRY",  "UNINFO_country", "UNINFO_titles", "First_Name", "Last_Name", "UNINFO.username", "UNINFO.email", "UNINFO.User Roles"]].sort_values(by="BOS_AF_user_info.COUNTRY")
df_result

,BOS_AF_user_info.COUNTRY,UNINFO_country,UNINFO_titles,First_Name,Last_Name,UNINFO.username,UNINFO.email,UNINFO.User Roles
20,Afghanistan,None,None,Catherine,MASAKA,NaN,NaN,none
85,Afghanistan,None,None,Hoda,EL SABBAHY,NaN,NaN,none
39,Afghanistan,"Niger,Nigeria","Agency Focal Point,CF Viewer,CF Editor",MARK,HUTCHINSON,hutchinson,hutchinson@unfpa.org,"Agency Focal Point - Nigeria, CF Editor - Nig..."
102,Albania,Albania,CF Viewer,Mirela,KUMBARO,mkumbaro,mkumbaro@unicef.org,CF Viewer - Albania
99,Albania,Albania,"Agency Focal Point,CF Viewer,CF Editor",Teuta,BERBERI,berberi,berberi@unhcr.org,"Agency Focal Point - Albania, CF Editor - Alb..."
...,...,...,...,...,...,...,...,...
1293,Yemen,None,None,Mulugeta,ABEBE,NaN,NaN,none
1309,Zambia,Zambia,"Agency Focal Point,CF Viewer",Roland,SERI,roland.seri_25,roland.seri@undp.org,"Agency Focal Point - Zambia, CF Viewer - Zambia"
1319,Zimbabwe,None,None,Carine,YENGAYENGE,NaN,NaN,none
1323,Zimbabwe,None,None,ESTHER,MUIA,NaN,NaN,none


In [13]:
'''
# drop the none entry values

df_result_nona = df_result.dropna(subset = ["extracted_country"])

df_result_nona
'''

'\n# drop the none entry values\n\ndf_result_nona = df_result.dropna(subset = ["extracted_country"])\n\ndf_result_nona\n'

In [14]:
df_result.fillna("None", inplace=True)
df_result["Existence_UNINFO"] = df_result["UNINFO.email"] != "None"

df_result

,BOS_AF_user_info.COUNTRY,UNINFO_country,UNINFO_titles,First_Name,Last_Name,UNINFO.username,UNINFO.email,UNINFO.User Roles,Existence_UNINFO
20,Afghanistan,None,None,Catherine,MASAKA,None,None,none,False
85,Afghanistan,None,None,Hoda,EL SABBAHY,None,None,none,False
39,Afghanistan,"Niger,Nigeria","Agency Focal Point,CF Viewer,CF Editor",MARK,HUTCHINSON,hutchinson,hutchinson@unfpa.org,"Agency Focal Point - Nigeria, CF Editor - Nig...",True
102,Albania,Albania,CF Viewer,Mirela,KUMBARO,mkumbaro,mkumbaro@unicef.org,CF Viewer - Albania,True
99,Albania,Albania,"Agency Focal Point,CF Viewer,CF Editor",Teuta,BERBERI,berberi,berberi@unhcr.org,"Agency Focal Point - Albania, CF Editor - Alb...",True
...,...,...,...,...,...,...,...,...,...
1293,Yemen,None,None,Mulugeta,ABEBE,None,None,none,False
1309,Zambia,Zambia,"Agency Focal Point,CF Viewer",Roland,SERI,roland.seri_25,roland.seri@undp.org,"Agency Focal Point - Zambia, CF Viewer - Zambia",True
1319,Zimbabwe,None,None,Carine,YENGAYENGE,None,None,none,False
1323,Zimbabwe,None,None,ESTHER,MUIA,None,None,none,False


In [15]:
# Country Check column

df_result.fillna("None", inplace=True)

df_result["Country_Check"] = df_result.apply(lambda x: x["BOS_AF_user_info.COUNTRY"] in x["UNINFO_country"], axis=1)

df_result[df_result["Country_Check"]==False]

,BOS_AF_user_info.COUNTRY,UNINFO_country,UNINFO_titles,First_Name,Last_Name,UNINFO.username,UNINFO.email,UNINFO.User Roles,Existence_UNINFO,Country_Check
20,Afghanistan,None,None,Catherine,MASAKA,None,None,none,False,False
85,Afghanistan,None,None,Hoda,EL SABBAHY,None,None,none,False,False
39,Afghanistan,"Niger,Nigeria","Agency Focal Point,CF Viewer,CF Editor",MARK,HUTCHINSON,hutchinson,hutchinson@unfpa.org,"Agency Focal Point - Nigeria, CF Editor - Nig...",True,False
136,Algeria,None,None,Federico,LO GIUDICE,federico.logiudice,federico.logiudice@un.org,,True,False
155,Angola,None,None,Vito,TRANI,None,None,none,False,False
...,...,...,...,...,...,...,...,...,...,...
1281,Venezuela,None,None,NELSON,MUNOZ,None,None,none,False,False
1293,Yemen,None,None,Mulugeta,ABEBE,None,None,none,False,False
1319,Zimbabwe,None,None,Carine,YENGAYENGE,None,None,none,False,False
1323,Zimbabwe,None,None,ESTHER,MUIA,None,None,none,False,False


In [18]:
df_result.to_excel("Preprocessed_data.xlsx", index=False)


# Count the number of users in BOS, UNINFO, and matched cases for RCO Checks

In [19]:
# count the number of people
#"Country_Check": "sum"
# , "Country_Check":"N_CountryMatch"

df_count = df_result.groupby("BOS_AF_user_info.COUNTRY").agg({
    "UNINFO.email": "count",
    "Existence_UNINFO":"sum",
    "Country_Check":"sum"
}).reset_index()

df_count.rename(columns = {"UNINFO.email":"N_BOS_Users", "Existence_UNINFO":"N_UNINFO_Users", "Country_Check":"N_Country_Match"}, inplace=True)

df_count

,BOS_AF_user_info.COUNTRY,N_BOS_Users,N_UNINFO_Users,N_Country_Match
0,Afghanistan,3,1,0
1,Albania,3,3,3
2,Algeria,2,2,1
3,Angola,3,1,1
4,Argentina,1,1,1
...,...,...,...,...
125,Venezuela,1,0,0
126,Viet Nam,3,3,3
127,Yemen,1,0,0
128,Zambia,1,1,1


In [20]:
# merge the dataset

df_merged = pd.merge(df_count, df_result, on =["BOS_AF_user_info.COUNTRY"] )

df_merged

,BOS_AF_user_info.COUNTRY,N_BOS_Users,N_UNINFO_Users,N_Country_Match,UNINFO_country,UNINFO_titles,First_Name,Last_Name,UNINFO.username,UNINFO.email,UNINFO.User Roles,Existence_UNINFO,Country_Check
0,Afghanistan,3,1,0,None,None,Catherine,MASAKA,None,None,none,False,False
1,Afghanistan,3,1,0,None,None,Hoda,EL SABBAHY,None,None,none,False,False
2,Afghanistan,3,1,0,"Niger,Nigeria","Agency Focal Point,CF Viewer,CF Editor",MARK,HUTCHINSON,hutchinson,hutchinson@unfpa.org,"Agency Focal Point - Nigeria, CF Editor - Nig...",True,False
3,Albania,3,3,3,Albania,CF Viewer,Mirela,KUMBARO,mkumbaro,mkumbaro@unicef.org,CF Viewer - Albania,True,True
4,Albania,3,3,3,Albania,"Agency Focal Point,CF Viewer,CF Editor",Teuta,BERBERI,berberi,berberi@unhcr.org,"Agency Focal Point - Albania, CF Editor - Alb...",True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
260,Yemen,1,0,0,None,None,Mulugeta,ABEBE,None,None,none,False,False
261,Zambia,1,1,1,Zambia,"Agency Focal Point,CF Viewer",Roland,SERI,roland.seri_25,roland.seri@undp.org,"Agency Focal Point - Zambia, CF Viewer - Zambia",True,True
262,Zimbabwe,3,0,0,None,None,Carine,YENGAYENGE,None,None,none,False,False
263,Zimbabwe,3,0,0,None,None,ESTHER,MUIA,None,None,none,False,False


In [21]:
'''

import numpy as np

df_merged["Status"] = np.where((df_merged["N_BOS_Users"]>2) | (df_merged["N_BOS_Users"] != df_merged["N_UNINFO_Users"]), "Review", "Confirm")

df_merged.loc[df_merged["Status"] == "Confirm", ["Status", "N_BOS_Users", "N_UNINFO_Users"]]

df_merged.loc[df_merged["Status"] == "Review", ["BOS_AF_user_info.COUNTRY", "Status", "N_BOS_Users", "N_UNINFO_Users"]]

'''

'\n\nimport numpy as np\n\ndf_merged["Status"] = np.where((df_merged["N_BOS_Users"]>2) | (df_merged["N_BOS_Users"] != df_merged["N_UNINFO_Users"]), "Review", "Confirm")\n\ndf_merged.loc[df_merged["Status"] == "Confirm", ["Status", "N_BOS_Users", "N_UNINFO_Users"]]\n\ndf_merged.loc[df_merged["Status"] == "Review", ["BOS_AF_user_info.COUNTRY", "Status", "N_BOS_Users", "N_UNINFO_Users"]]\n\n'

In [22]:


df_merged.loc[(df_merged["N_BOS_Users"] != df_merged["N_UNINFO_Users"]) & (df_merged["N_BOS_Users"] < 3), "Comment" ] = "The number of BOS and UNINFO users does not match"

df_merged.loc[(df_merged["N_UNINFO_Users"] > df_merged["N_Country_Match"] ), "Comment" ] = "Need to validate the usesr info"

df_merged.loc[df_merged["N_BOS_Users"] >= 3, "Comment"] = "Too many users in BOS"

df_merged.loc[df_merged["N_UNINFO_Users"]==0, "Comment"] = "No Users in the UNINFO System"

# df_merged.loc[(df_merged["N_UNINFO_Users"] != df_merged["N_Country_Match"]) & (df_merged["N_BOS_Users"] < 3), "Comment" ] = "Need to validate UNINFO Users' infromation"

df_merged["Comment"].value_counts()

Too many users in BOS                                87
No Users in the UNINFO System                        67
The number of BOS and UNINFO users does not match    42
Need to validate the usesr info                      15
Name: Comment, dtype: int64

In [ ]:
df_merged[df_merged["Comment"] == "None"]

,BOS_AF_user_info.COUNTRY,N_BOS_Users,N_UNINFO_Users,N_Country_Match,UNINFO_country,UNINFO_titles,First_Name,Last_Name,UNINFO.username,UNINFO.email,UNINFO.User Roles,Existence_UNINFO,Country_Check,Comment


In [23]:
df_merged.loc[df_merged["Comment"] == "No Users in the UNINFO System" , "Urgency"] = "Severe"

df_merged.loc[df_merged["Comment"] == "Too many users in BOS" , "Urgency"] = "High"

df_merged.loc[df_merged["Comment"] == "The number of BOS and UNINFO users does not match" , "Urgency"] = "Medium"
df_merged.loc[df_merged["Comment"] == "Need to validate the usesr info" , "Urgency"] = "Medium"

df_merged.loc[(df_merged["N_BOS_Users"]==1) & (df_merged["N_UNINFO_Users"]==1 )& (df_merged["N_Country_Match"]==1), "Urgency"] = "Low"
df_merged.loc[(df_merged["N_BOS_Users"]==2) & (df_merged["N_UNINFO_Users"]==2 )& (df_merged["N_Country_Match"]==2), "Urgency"] = "Low"

df_merged.loc[df_merged["Comment"] == "None", "Urgency"] = "Low"

df_merged["Urgency"].value_counts()

High      87
Severe    67
Medium    57
Low       54
Name: Urgency, dtype: int64

In [24]:
len(df_merged)

87+67+57+54

265

In [25]:
df_merged["Comment"].fillna("None", inplace=True)

In [ ]:
'''
df_merged_columns = df_merged.columns

new_columns = [df_merged_columns[0]] + df_merged_columns[-2:].tolist() + df_merged_columns[1:-2].tolist()

df_final = df_merged[new_columns]
'''

'\ndf_merged_columns = df_merged.columns\n\nnew_columns = [df_merged_columns[0]] + df_merged_columns[-2:].tolist() + df_merged_columns[1:-2].tolist()\n\ndf_final = df_merged[new_columns]\n'

In [26]:
# Add region information

df_region = pd.read_excel("/content/gdrive/MyDrive/Data/UN_USER/20231010_BOS_UNINFO_priority_users.xlsx",
                           sheet_name = "1.b. Heads of RCO-Details")

df_region = df_region[["Region", "Country"]]

df_region_merged  = pd.merge(df_region, df_merged, left_on =["Country"], right_on = ["BOS_AF_user_info.COUNTRY"] )

df_region_merged.drop("BOS_AF_user_info.COUNTRY", axis=1, inplace=True)


df_region_merged


,Region,Country,N_BOS_Users,N_UNINFO_Users,N_Country_Match,UNINFO_country,UNINFO_titles,First_Name,Last_Name,UNINFO.username,UNINFO.email,UNINFO.User Roles,Existence_UNINFO,Country_Check,Comment,Urgency
0,APA,Afghanistan,3,1,0,None,None,Catherine,MASAKA,None,None,none,False,False,Too many users in BOS,High
1,APA,Afghanistan,3,1,0,None,None,Hoda,EL SABBAHY,None,None,none,False,False,Too many users in BOS,High
2,APA,Afghanistan,3,1,0,"Niger,Nigeria","Agency Focal Point,CF Viewer,CF Editor",MARK,HUTCHINSON,hutchinson,hutchinson@unfpa.org,"Agency Focal Point - Nigeria, CF Editor - Nig...",True,False,Too many users in BOS,High
3,AFR,Algeria,2,2,1,Algeria,"Agency Focal Point,CF Viewer",Donald,KARERWA,dkarerwa,dkarerwa@iom.int,"Agency Focal Point - Algeria, CF Viewer - Alg...",True,True,Need to validate the usesr info,Medium
4,AFR,Algeria,2,2,1,None,None,Federico,LO GIUDICE,federico.logiudice,federico.logiudice@un.org,,True,False,Need to validate the usesr info,Medium
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
278,AS,Yemen,1,0,0,None,None,Mulugeta,ABEBE,None,None,none,False,False,No Users in the UNINFO System,Severe
279,AFR,Zambia,1,1,1,Zambia,"Agency Focal Point,CF Viewer",Roland,SERI,roland.seri_25,roland.seri@undp.org,"Agency Focal Point - Zambia, CF Viewer - Zambia",True,True,None,Low
280,AFR,Zimbabwe,3,0,0,None,None,Carine,YENGAYENGE,None,None,none,False,False,No Users in the UNINFO System,Severe
281,AFR,Zimbabwe,3,0,0,None,None,ESTHER,MUIA,None,None,none,False,False,No Users in the UNINFO System,Severe


# Save the File to Excel and Download it

In [ ]:
filename = 'OMT_Summary_2023-11-01.xlsx'

df_region_merged.to_excel(filename, index=False)

#from google.colab import files  # Import the files module
#files.download(filename)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>